# Projeto 2 - Ciência dos Dados

Nome: Matheus Freitas Sant'Ana

Nome: Thiago Lopes David

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [61]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [62]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [63]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [64]:
#Produto escolhido:
produto = 'uber'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [65]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [66]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [67]:
import os
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\matfs\Desktop\Ciência dos Dados\P2-Ciencia-dos-Dados


In [68]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [69]:
tweets = pd.read_excel('uber.xlsx')

In [70]:
DataFrame_irrelevantes = tweets.loc[tweets['Classificação'] == 0]
contagem_irrel = pd.Series(cleanup(" ".join(DataFrame_irrelevantes["Treinamento"])).split()).value_counts()
contagem_irrel

uber                157
o                   110
um                   83
de                   79
e                    78
eu                   75
aí                   57
rt                   55
que                  47
a                    42
no                   38
tava                 38
do                   36
hoje                 31
pegar                30
com                  30
na                   29
fui                  28
3                    27
pra                  27
eh                   27
cristão              27
ne                   27
não                  27
sou                  27
sim                  26
louvorzao            26
senhor               26
(uns                 26
@brukopinho          26
                   ... 
@crisenoflamengo      1
usava                 1
fumar                 1
phoebe                1
esquina               1
autônomo              1
homem                 1
moça                  1
@beatriz_seca         1
vomita                1
leiteiros       

In [71]:
DataFrame_relevantes = tweets.loc[tweets['Classificação'] == 1]
contagem_rel = pd.Series(cleanup(" ".join(DataFrame_relevantes["Treinamento"])).split()).value_counts()
contagem_rel

uber             126
de                77
o                 69
e                 68
que               54
eu                54
a                 49
pra               45
um                41
no                35
q                 32
não               28
do                27
com               25
meu               21
minha             21
é                 21
na                19
da                18
ele               18
me                16
uma               16
https             15
//t               15
rt                14
vai               13
se                13
mais              13
tá                12
em                11
                ... 
complicado         1
puxar              1
esqueceu           1
netflix            1
playlist           1
adormeço           1
@vitorcugler       1
outra,             1
virando            1
carro,             1
*seu               1
aprende            1
raiva              1
terminando         1
primeiro           1
mais,              1
igual        

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**